In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp auto

In [ ]:
#| export
from typing import Callable, List, Optional

import numpy as np
import optuna
import pandas as pd
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from utilsforecast.compat import DataFrame
from utilsforecast.losses import smape

from mlforecast import MLForecast
from mlforecast.core import Freq, Models, _get_model_name, _name_models
from mlforecast.optimization import mlforecast_objective
from mlforecast.target_transforms import Differences, LocalStandardScaler, GlobalSklearnTransformer

In [ ]:
#| exporti
_log1p_tfm = GlobalSklearnTransformer(
    FunctionTransformer(func=np.log1p, inverse_func=np.expm1)
)

def _seasonality_based_config(season_length: int):
    def config(trial):
        candidate_targ_tfms = [
            [LocalStandardScaler()],
            [Differences([1]), LocalStandardScaler()],
            [_log1p_tfm, LocalStandardScaler()],
            [_log1p_tfm, Differences([1]), LocalStandardScaler()],
        ]
        # if season_length > 1:
        #     candidate_targ_tfms.extend(
        #         [
        #             [_log1p_tfm, Differences([1, season_length]), LocalStandardScaler()],
        #             [Differences([1, season_length]), LocalStandardScaler()],
        #         ]
        #     )
        max_lags = trial.suggest_int('max_lags', 1, season_length)
        targ_tfms_idx = trial.suggest_categorical(
            'target_transforms_idx', range(len(candidate_targ_tfms))
        )
        return {
            'lags': range(1, max_lags + 1),
            'target_transforms': candidate_targ_tfms[targ_tfms_idx],
        }

    return config

def _lightgbm_space(trial):
    return {
        "n_estimators": trial.suggest_int("n_estimators", 20, 1000, log=True),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 1),
        "objective": trial.suggest_categorical("objective", ['l1', 'l2']),
        "verbosity": -1,
    }
    
def _xgboost_space(trial):
    return {
        "n_estimators": trial.suggest_int("n_estimators", 20, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.2, log=True),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'bagging_freq': trial.suggest_float('bagging_freq', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'min_data_in_leaf': trial.suggest_float('min_data_in_leaf', 1, 100),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 10),
    }
    
def _catboost_space(trial):
    return {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        'depth': trial.suggest_int('depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.2, log=True),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1.0),
        'min_data_in_leaf': trial.suggest_float('min_data_in_leaf', 1, 100),
    }
    
def _linear_regression_space(trial):
    return {
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False])
    }
    
def _ridge_space(trial):
    return {
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        'alpha': trial.suggest_float('alpha', 0.001, 10.0)
    }
    
def _lasso_space(trial):
    return {
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        'alpha': trial.suggest_float('alpha', 0.001, 10.0)
    }
    
def _elasticnet_space(trial):
    return {
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        'alpha': trial.suggest_float('alpha', 0.001, 10.0),
        'l1_ratio': trial.suggest_float('l1_ratio', 0.0, 1.0)
    }

def _random_forest_space(trial):
    return {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_samples_split': trial.suggest_int('min_child_samples', 1, 100),
        'max_features': trial.suggest_float('max_features', 0.5, 1.0),
        "criterion": trial.suggest_categorical("criterion", ['squared_error', 'poisson']),
    }

_default_model_spaces = {
    'LGBMRegressor': _lightgbm_space,
    'XGBRegressor': _xgboost_space,
    'CatBoostRegressor': _catboost_space,
    'LinearRegression': _linear_regression_space,
    'Ridge': _ridge_space,
    'Lasso': _lasso_space,
    'ElasticNet': _elasticnet_space,
    'RandomForest': _random_forest_space,
}

In [ ]:
#| export
class AutoMLForecast:

    def __init__(
        self,
        models: Models,        
        freq: Freq,
        season_length: int,
        model_configs: Optional[List[Callable]] = None,
        init_config: Optional[Callable] = None,
        fit_config: Optional[Callable] = None,
    ):
        self.freq = freq
        if not isinstance(models, dict) and not isinstance(models, list):
            models = [models]
        if isinstance(models, list):
            model_names = _name_models([_get_model_name(m) for m in models])
            models_with_names = dict(zip(model_names, models))
        else:
            models_with_names = models
        self.models = models_with_names
        if model_configs is not None:
            self.model_configs = model_configs
        else:
            self.model_configs = len(self.models) * [None]
        for i, (name, config) in enumerate(zip(self.models.keys(), self.model_configs)):
            if config is not None:
                continue
            if name not in _default_model_spaces:
                raise NotImplementedError(
                    f"{name} does not have a default config. Please provide one."
                )
            self.model_configs[i] = _default_model_spaces[name]
        if init_config is not None:
            self.init_config = init_config
        else:
            self.init_config = _seasonality_based_config(season_length)
        if fit_config is not None:
            self.fit_config = fit_config
        else:
            self.fit_config = lambda trial: {}

    def fit(
        self,
        df: DataFrame,
        n_windows: int,
        h: int,
        num_samples: int,        
        loss: Optional[Callable] = None,
        search_alg: Optional[optuna.samplers.BaseSampler] = None,
        storage: Optional[optuna.storages.BaseStorage] = None,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
    ) -> 'AutoMLForecast':
        if loss is None:
            def loss(df):
                return smape(df, models=['model'])['model'].mean()
        if search_alg is None:
            search_alg = optuna.samplers.TPESampler(seed=0)

        self.results_ = []
        self.models_ = {}
        for (name, model), model_config in zip(self.models.items(), self.model_configs):
            def config_fn(trial: optuna.Trial) -> float:
                return {
                    'model_params': model_config(trial),
                    'mlf_init_params': self.init_config(trial),
                    'mlf_fit_params': self.fit_config(trial)
                }

            objective = mlforecast_objective(
                df=df,
                config_fn=config_fn,
                eval_fn=loss,
                model=model,
                freq=self.freq,
                n_windows=n_windows,
                h=h,
                id_col=id_col,
                time_col=time_col,
                target_col=target_col,
            )
            study = optuna.create_study(
                direction='minimize',
                sampler=search_alg,
                storage=storage,
            )
            study.optimize(objective, n_trials=num_samples, n_jobs=1)
            self.results_.append(study)
            best_config = study.best_trial.user_attrs['config']            
            best_model = clone(model)
            best_model.set_params(**best_config['model_params'])
            self.models_[name] = MLForecast(
                models={name: best_model},
                freq=self.freq,
                **best_config['mlf_init_params']
            )
            self.models_[name].fit(
                df,
                **best_config['mlf_fit_params']
            )
        return self

    def predict(
        self,
        h: int,
        X_df: Optional[DataFrame] = None,
    ) -> DataFrame:
        all_preds = None
        for name, model in self.models_.items():
            preds = model.predict(h=h, X_df=X_df)
            if all_preds is None:
                all_preds = preds
            else:
                all_preds[name] = preds[name]
        return all_preds

In [ ]:
import lightgbm as lgb
from datasetsforecast.m4 import M4, M4Evaluation, M4Info
from sklearn.linear_model import Ridge
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def train_valid_split(group):
    df, *_ = M4.load(directory='data', group=group)
    df['ds'] = df['ds'].astype('int')
    horizon = M4Info[group].horizon
    valid = df.groupby('unique_id').tail(horizon)
    train = df.drop(valid.index)
    return train, valid

In [ ]:
h = M4Info['Weekly'].horizon
weekly_train, weekly_valid = train_valid_split('Weekly')
weekly_train['unique_id'] = weekly_train['unique_id'].astype('category')
weekly_valid['unique_id'] = weekly_valid['unique_id'].astype(weekly_train['unique_id'].dtype)

In [ ]:
auto_mlf = AutoMLForecast(
    freq=1,
    season_length=52,
    models=[
        lgb.LGBMRegressor(),
        make_pipeline(
            ColumnTransformer(
                [('encoder', OneHotEncoder(), ['unique_id'])],
                remainder='passthrough',
            ),
            Ridge(),
        ),
    ],
    model_configs=[
        None,
        lambda trial: {f'ridge__{k}': v for k, v in _ridge_space(trial).items()},
    ],
    fit_config=lambda trial: {'static_features': ['unique_id']}
)

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
auto_mlf.fit(
    df=weekly_train,
    n_windows=2,
    h=h,
    num_samples=20,
)

<__main__.AutoMLForecast>

In [ ]:
preds = auto_mlf.predict(h=h)

In [ ]:
for model in auto_mlf.models.keys():
    print(model)
    print(M4Evaluation.evaluate('data', 'Weekly', preds[model].values.reshape(-1, 13)))

LGBMRegressor
           SMAPE      MASE       OWA
Weekly  7.884069  2.179517  0.822674
Ridge
           SMAPE      MASE       OWA
Weekly  9.167321  2.543791  0.958291


In [ ]:
[res.best_trial.user_attrs['config'] for res in auto_mlf.results_]

[{'model_params': {'n_estimators': 100,
   'lambda_l1': 0.00199167444491787,
   'lambda_l2': 1.9488396642196672e-07,
   'num_leaves': 456,
   'feature_fraction': 0.8692518074746102,
   'bagging_fraction': 0.5080421148439958,
   'bagging_freq': 1,
   'objective': 'l1',
   'verbosity': -1},
  'mlf_init_params': {'lags': range(1, 48),
   'target_transforms': [<mlforecast.target_transforms.GlobalSklearnTransformer>,
    <mlforecast.target_transforms.Differences>,
    <mlforecast.target_transforms.LocalStandardScaler>]},
  'mlf_fit_params': {'static_features': ['unique_id']}},
 {'model_params': {'ridge__fit_intercept': True,
   'ridge__alpha': 5.398556881750432},
  'mlf_init_params': {'lags': range(1, 2),
   'target_transforms': [<mlforecast.target_transforms.GlobalSklearnTransformer>,
    <mlforecast.target_transforms.Differences>,
    <mlforecast.target_transforms.LocalStandardScaler>]},
  'mlf_fit_params': {'static_features': ['unique_id']}}]